## Functions Notebook

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import category_encoders as ce 
from sklearn.model_selection import KFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [3]:
# Create image tag
imageTag = [f'image{i:03d}' for i in range(200)]

# function that saves visualisation plots
def save_plot_function(imageRef, folder=r'C:\Users\hjame\Documents\Data Science\Mini Projects\capstoneProject\images'):
    filename = f'{imageTag[imageRef]}.png'
    path = os.path.join(folder, filename)
    plt.savefig(path, facecolor = 'w', bbox_inches='tight',pad_inches=0.3, transparent=True)
    plt.show()
    print(f'Plot saved to: {path}')
    return

# create function to show how the target variable varies based on other columns
def target_distribution_barplot(dataframe, x_feature, y_feature, imageNum, palette='Set1'):
    # plt.figure(figsize=(14,8))
    sns.barplot(data=dataframe, 
                x=x_feature, 
                y=y_feature, 
                palette=palette, 
                estimator=sum,
                errorbar=None,
                hue=x_feature, 
                legend=False ).set(
    title=f'Fraudulent Job Postings by {x_feature}')
    plt.ylabel('Total of Fraudulent Job Postings')
    plt.xticks(rotation=45)
    save_plot_function(imageNum)
    return

# create function to identify text that is not english
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

# text preprocessing function
# convert to lowercase, remove text in sqaure brackets, remove links, remove punctuation, remove words containing numbers 

def clean_text(text):
    text = text.lower() # Lowercase
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'http\S+|https\S+|www\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    words = nltk.word_tokenize(text)
    words = [lemma.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)  

# learning curve function for three models 

def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

# Word2Vec function 
# convert each document into single vector by averaging word emeddings

def get_average_word2vec(tokens, model, vector_size):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(vector_size)
    return np.mean(model.wv[valid_tokens], axis=0)

# learning curve function 
def learning_curve_function(model, X, y, imageNum, name='Model', cv=5, scoring='accuracy'):
    # Get learning curve data
    train_sizes, train_scores, test_scores = learning_curve(
        estimator=model,
        X=X,
        y=y,
        cv=cv,
        scoring=scoring,
        train_sizes=np.linspace(0.1, 1.0, 10),
        n_jobs=-1,
        shuffle=True,
        random_state=42
    )

    # Calculate mean and standard deviation
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, 'o-', color='green', label='Training Score')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='green')

    plt.plot(train_sizes, test_mean, 'o-', color='red', label='Cross-validation Score')
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')

    plt.title(f'Learning Curve: {name}', fontsize=14)
    plt.xlabel('Training Set Size', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.legend()

    # Save figure
    save_plot_function(imageNum)

    plt.show()

# # Randomized hyper parameter tuning function 

# def hyper_tune_evalutation_model(model, param_dist, X_train, X_test, y_train, y_test, model_name, vectorizer_name, n_iter=20, cv=5):
#     random_search = RandomizedSearchCV(
#         estimator=model,
#         param_distribution,
#         n_iter=n_iter,
#         cv=cv,
#         scoring='accuracy',
#         n_job=-1,
#         random_state=42,
#     )

#     random_search.fit(X_train, y_train)
#     best_model = random_search.best_estimator_
#     y_pred = best_model.predict(X_test)
#     roc = roc_auc_score(y_test, best_model_predict_proba(X_test)[:,1])

#     results.append({
#     'Model': model_name,
#     'Vectorizer': Vectorizer_name,
#     'Best_Params': random_search.best_params,
#     'Test Accuracy': best_model.score(X_test, y_test),
#     'ROC AUC': roc
#     })

#     print(f'Model: {model_name}')
#     print(f'Vectorizer: {vectorizer_name}')
#     print(f'Best Parameters: {random_search.best_params_}')
#     print(f'Confusion Matrix:\n, {confusion_matrix(y_test, y_pred)}')
#     print(f'Classification Report:\n, {classification_report(y_test, y_pred)}')
    